# Capstone Two - 2 Data wrangling<a id='2'></a>

## 2.1 Contents<a id='2.1'></a>
* [2 Data wrangling](#2)
  * [2.1 Contents](#2.1)
  * [2.2 Imports](#2.2)
  * [2.3 Load The Store Sales Data](#2.3)
  * [2.4 Explore The Data](#2.4)
    * [2.4.1 Target](#2.4.1)  
    * [2.4.2 Dimensions and Number of Time Series](#2.4.2)
    * [2.4.3 Categorical Exogenous Variables](#2.4.3)
      * [2.4.3.1 Store Metadata](#2.4.3.1)
        * [2.4.3.1.1 Cluster](#2.4.3.1.1)
        * [2.4.3.1.2 City and State](#2.4.3.1.2)
        * [2.4.3.1.3 Type](#2.4.3.1.3)
      * [2.4.3.2 Holidays and Events](#2.4.3.2)
        * [2.4.3.2.1 Drop Duplicate Dates](#2.4.3.2.1)
        * [2.4.3.2.2 Holiday/Event Types](#2.4.3.2.2)
    * [2.4.4 Numeric Exogenous Variables](#2.4.4)
      * [2.4.4.1 Daily Oil Price](#2.4.4.1)
      * [2.4.4.2 Daily Transactions Per Store](#2.4.4.2)
      * [2.4.4.3 Quantity of On-Promotion Product](#2.4.4.3)
  * [2.5 Save data](#2.5)
  * [2.6 Summary](#2.6)

## 2.2 Imports<a id='2.2'></a>

In [1]:
import pandas as pd

from library.sb_utils import save_file

## 2.3 Load The Store Sales Data<a id='2.3'></a>

Download the store sales data and other supporting data from this webpage: https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data

In [2]:
# Directory of input data
input_dir = '../input_data/store-sales-time-series-forecasting/'

In [3]:
# Store sales (training set)
train = pd.read_csv(input_dir+'train.csv')

# Store sales (testing set)
test = pd.read_csv(input_dir+'test.csv')

# Store metadata
stores = pd.read_csv(input_dir+'stores.csv')

# Daily oil price
oil = pd.read_csv(input_dir+'oil.csv')

# Number of daily transactions at a store
transactions = pd.read_csv(input_dir+'transactions.csv')

# Holidays and Events, with metadata
holidays_events = pd.read_csv(input_dir+'holidays_events.csv')

## 2.4 Explore The Data<a id='2.4'></a>

### 2.4.1 Target<a id='2.4.1'></a>

In [4]:
train.head(3)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [6]:
test.head(3)

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2


`sales` gives the total units sold for a product family at a particular store at a given date. Its time series is our **target** to be modeled, and it's a **non-categorical** variable.

`date` is the time column.

`onpromotion` is one of the potential **Exogenous Variables**, giving the total number of items in a product family that were being promoted a store at a given date.

### 2.4.2 Dimensions and Number of Time Series<a id='2.4.2'></a>

The two dimensions with this project are "store" and "product family":

In [7]:
train[['store_nbr','family']].nunique()

store_nbr    54
family       33
dtype: int64

In [8]:
test[['store_nbr','family']].nunique()

store_nbr    54
family       33
dtype: int64

We have 54 stores and 33 product families, thus totally **54*33=1782** time series.

In [9]:
train[['store_nbr','family']].value_counts()

store_nbr  family                 
1          AUTOMOTIVE                 1684
36         PREPARED FOODS             1684
37         CLEANING                   1684
           CELEBRATION                1684
           BREAD/BAKERY               1684
                                      ... 
18         PREPARED FOODS             1684
           POULTRY                    1684
           PLAYERS AND ELECTRONICS    1684
           PET SUPPLIES               1684
54         SEAFOOD                    1684
Length: 1782, dtype: int64

In [10]:
train.tail(3)

,id,date,store_nbr,family,sales,onpromotion
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0


In [11]:
test[['store_nbr','family']].value_counts()

store_nbr  family                 
1          AUTOMOTIVE                 16
36         PREPARED FOODS             16
37         CLEANING                   16
           CELEBRATION                16
           BREAD/BAKERY               16
                                      ..
18         PREPARED FOODS             16
           POULTRY                    16
           PLAYERS AND ELECTRONICS    16
           PET SUPPLIES               16
54         SEAFOOD                    16
Length: 1782, dtype: int64

In [12]:
test.tail(3)

,id,date,store_nbr,family,onpromotion
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9
28511,3029399,2017-08-31,9,SEAFOOD,0


**Resolution** of Time Series: **Daily**

Each time series of the training data has 1684 days (01-01-2013 to 2017-08-15).

Each time series of the testing data has 16 days (2017-08-16 to 2017-08-31).

**Forecasting horizon: 16 days**

### 2.4.3 Categorical Exogenous Variables<a id='2.4.3'></a>

#### 2.4.3.1 Store Metadata<a id='2.4.3.1'></a>

In [13]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [14]:
stores.info()
stores.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


store_nbr    54
city         22
state        16
type          5
cluster      17
dtype: int64

We can see that `city`, `state`, and `type` of the store are **categorical** features, while the data type of `cluster` is integer.

##### 2.4.3.1.1 Cluster<a id='2.4.3.1.1'></a>

In [15]:
sorted(stores['cluster'].unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [16]:
stores['cluster'].value_counts()

3     7
6     6
10    6
15    5
13    4
14    4
11    3
4     3
8     3
1     3
9     2
7     2
2     2
12    1
5     1
16    1
17    1
Name: cluster, dtype: int64

Although `cluster` is a integer column, it's still a **categorical** feature and it has 17 categories (1-17).

##### 2.4.3.1.2 City and State<a id='2.4.3.1.2'></a>

In [17]:
stores['city'].value_counts()

Quito            18
Guayaquil         8
Cuenca            3
Santo Domingo     3
Manta             2
Latacunga         2
Machala           2
Ambato            2
Quevedo           1
Esmeraldas        1
Loja              1
Libertad          1
Playas            1
Daule             1
Babahoyo          1
Salinas           1
Puyo              1
Guaranda          1
Ibarra            1
Riobamba          1
Cayambe           1
El Carmen         1
Name: city, dtype: int64

In [18]:
stores['state'].value_counts()

Pichincha                         19
Guayas                            11
Santo Domingo de los Tsachilas     3
Azuay                              3
Manabi                             3
Cotopaxi                           2
Tungurahua                         2
Los Rios                           2
El Oro                             2
Chimborazo                         1
Imbabura                           1
Bolivar                            1
Pastaza                            1
Santa Elena                        1
Loja                               1
Esmeraldas                         1
Name: state, dtype: int64

Most of the stores are located in the cities of "Quito" and "Guayaquil" and the states of "Pichincha" and "Guayas".

In [19]:
st_pi=stores[stores['state'] == 'Pichincha']
st_pi[st_pi['city'] != 'Quito']

,store_nbr,city,state,type,cluster
10,11,Cayambe,Pichincha,B,6


In [20]:
st_gu=stores[stores['state'] == 'Guayas']
st_gu[st_gu['city'] != 'Guayaquil']

,store_nbr,city,state,type,cluster
26,27,Daule,Guayas,D,1
34,35,Playas,Guayas,C,3
35,36,Libertad,Guayas,E,10


There is one store located in the state of "Pichincha" but not in the city of "Quito".\
There are three store located in the state of "Guayas" but not in the city of "Guayaquil".

In [21]:
(stores['city'] != stores['state']).value_counts()

True     52
False     2
dtype: int64

In [22]:
stores[stores['city'] == stores['state']]

,store_nbr,city,state,type,cluster
37,38,Loja,Loja,D,4
42,43,Esmeraldas,Esmeraldas,E,10


There are two stores with the same city and state name.

##### 2.4.3.1.3 Type<a id='2.4.3.1.3'></a>

In [23]:
stores['type'].value_counts()

D    18
C    15
A     9
B     8
E     4
Name: type, dtype: int64

There are 5 types of stores and most of the stores are of type "D" and "C".

#### 2.4.3.2 Holidays and Events<a id='2.4.3.2'></a>

In [24]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [25]:
# Only select the time period we need
holidays_events_tmp = holidays_events.set_index("date").sort_index()
holidays_events_sel = holidays_events_tmp.loc['2013-01-01': '2017-08-31'].reset_index()

In [26]:
# Only taking "National" holiday or events for simplicity
holidays_events_sel = holidays_events_sel[holidays_events_sel['locale'] == 'National'] 

In [27]:
# Drop the "description", "locale", and "locale_name" columns
holidays_events_sel.drop(columns=['locale', 'locale_name','description'], inplace=True)

In [28]:
holidays_events_sel.head()

,date,type,transferred
0,2013-01-01,Holiday,False
1,2013-01-05,Work Day,False
2,2013-01-12,Work Day,False
3,2013-02-11,Holiday,False
4,2013-02-12,Holiday,False


In [29]:
holidays_events_sel.nunique()

date           147
type             6
transferred      2
dtype: int64

##### 2.4.3.2.1 Drop Duplicate Dates<a id='2.4.3.2.1'></a>

In [30]:
# Find dates with more than one holidays/events
holidays_events_sel['date'].value_counts().head()

2016-05-01    2
2016-05-08    2
2016-05-07    2
2014-12-26    2
2016-04-29    1
Name: date, dtype: int64

In [31]:
# Check holiday types on duplicate dates
holidays_events_sel[holidays_events_sel['date'].isin(['2016-05-01','2016-05-08','2016-05-07','2014-12-26'])]

,date,type,transferred
115,2014-12-26,Bridge,False
116,2014-12-26,Additional,False
194,2016-05-01,Holiday,False
195,2016-05-01,Event,False
201,2016-05-07,Additional,False
202,2016-05-07,Event,False
203,2016-05-08,Event,False
204,2016-05-08,Event,False


We can see there are no transferred holidays on these duplicated dates, so we can apply the `drop_duplicates` method to the 'date' column.

In [32]:
holidays_events_sel.drop_duplicates(subset=['date'], inplace=True)

In [33]:
# Check if no duplicate dates now
holidays_events_sel['date'].value_counts().head()

2013-01-01    1
2016-05-10    1
2016-04-24    1
2016-04-25    1
2016-04-26    1
Name: date, dtype: int64

##### 2.4.3.2.2 Holiday/Event Types<a id='2.4.3.2.2'></a>

In [34]:
holidays_events_sel['type'].value_counts()

Event         53
Holiday       51
Additional    28
Transfer       7
Work Day       5
Bridge         3
Name: type, dtype: int64

In [35]:
holidays_events_sel['transferred'].value_counts()

False    140
True       7
Name: transferred, dtype: int64

There are several types of holidays we need to notice:
1. Transfer: Linked with the "transferred" column. The "Holiday"/"Event" date with the "transferred" column equals "True" was more like a normal day, while the date with the type of "Transfer" was the day actually celebrated.
2. Bridge: Extra days that were added to a holiday.
3. Work Day: Days not normally scheduled for work (e.g., Saturday) that were meant to payback the Bridge.
4. Additional: Days added a regular calendar holiday, for example, as typically happens around Christmas.

**Summary:** "Holiday" or "Event" with "transferred" column = True were not holidays; "Transfer", "Bridge" and "Additional" were holidays; "Work Day" were not holidays.

### 2.4.4 Numerical Exogenous Variables<a id='2.4.4'></a>

#### 2.4.4.1 Daily Oil Price<a id='2.4.4.1'></a>

In [36]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [37]:
oil.tail()

,date,dcoilwtico
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96
1217,2017-08-31,47.26


We can already see that there are missing value and time gaps in the oil price column.

#### 2.4.4.2 Daily Transactions Per Store<a id='2.4.4.2'></a>

In [38]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


On 2013-01-01, only one of the 54 stores had transactions.

In [39]:
transactions.tail()

,date,store_nbr,transactions
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932
83487,2017-08-15,54,802


"transactions" were directly related to "sales", so we didn't have the "transactions" data for the forecasting period.

#### 2.4.4.3 Quantity of On-Promotion Product<a id='2.4.4.3'></a>

In [40]:
train['onpromotion'].head()

0    0
1    0
2    0
3    0
4    0
Name: onpromotion, dtype: int64

In [41]:
train['onpromotion'].tail()

3000883      0
3000884      1
3000885    148
3000886      8
3000887      0
Name: onpromotion, dtype: int64

In [42]:
test['onpromotion'].tail()

28507    1
28508    0
28509    1
28510    9
28511    0
Name: onpromotion, dtype: int64

## 2.5 Save Data<a id='2.5'></a>

In [43]:
# save the cleaned holidays_events data to a new csv file
datapath = '../data'
save_file(holidays_events_sel, 'holidays_events_cleaned.csv', datapath)

Writing file.  "../data/holidays_events_cleaned.csv"


## 2.6 Summary<a id='2.6'></a>

The "sales" variables has been identified as the target feature (non-categorical). 

There are 54 stores and 33 product families, thus totally 1782 time series.

The resolution of time series is daily, and the forecasting horizon is 16 days.

In addition to time and the target, there are some Exogenous Variables:
1. Categorical features: store's city, state, type, and cluster; whether the date is a holiday or not.
2. Numerical Features: daily oil price; daily transactions per store; quantity of on-promotion products.